<a href="https://colab.research.google.com/github/jyizheng/ACM/blob/master/leetcode/PersistKV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def say_hello():
    print('Hello, World')

for i in range(5):
    say_hello()

class Btree:
    def __init__(self, cache_size, node_size):
        self.dict = {}

    def put(self, key, val):
        if not key in self.dict:
            self.dict[key] = val
        self.dict[key] = val

    def get(self, key):
        if not key in self.dict:
            return None
        return self.dict[key]
    def delete(self, key):
        if not key in self.dict:
            return
        del self.dict[key]

import os
class LogManager:
    def __init__(self, file_name):
        if os.path.exists(file_name):
            print(f"file exists")
        else:
            self.log_fd = open(file_name, 'wb')

        self.file_name = file_name

    def write_log_entry(self, key, val):
        four_bytes_from_key = key.to_bytes(4, byteorder='big')
        self.log_fd.write(four_bytes_from_key)

        four_bytes_from_val = val.to_bytes(4, byteorder='big')
        self.log_fd.write(four_bytes_from_val)
        self.log_fd.flush()
        os.fsync(self.log_fd.fileno())

        file_size = os.path.getsize('btree.log')
        print(f"file_size after logging={file_size}")

    def replay(self, mem_dict):
        file_size = os.path.getsize('btree.log')
        read_size = 0

        log_fd = open(self.file_name, 'rb')
        print(f"file_size replay={file_size}")

        while read_size < file_size:
            key_bytes = log_fd.read(4)
            value_bytes = log_fd.read(4)
            key = int.from_bytes(key_bytes, byteorder='big')
            value = int.from_bytes(value_bytes, byteorder='big')

            print(f"key={key}, value={value}")

            if value == (2<<31 -1):
                del mem_dict.dict[key]
            else:
                mem_dict.dict[key] = value

            read_size += 8


class Cache:
    def __init__(self, cache_size, node_size):
        self.mem_btree = Btree(cache_size, node_size)
        self.logger = LogManager('btree.log')
        #self.on_disk_btree = open('btree_page.data')
        self.recover()

    def recover(self):
        file_size = os.path.getsize('btree.log')
        print(f"file_size recover={file_size}")

        if file_size == 0:
            return

        self.logger.replay(self.mem_btree)



    def put(self, key, val):
        self.logger.write_log_entry(key, val)
        self.mem_btree.put(key, val)


    def get(self, key):
        return self.mem_btree.get(key)

    def delete(self, key):
        self.logger.write_log_entry(key, 2<<31 -1)
        self.mem_btree.delete(key)



cache = Cache(0, 0)
cache.put(1, 10)
val = cache.get(1)
assert val == 10

cache_new = Cache(0, 0)
val = cache_new.get(1)
assert val == 10


Hello, World
Hello, World
Hello, World
Hello, World
Hello, World
file_size recover=0
file_size after logging=8
file exists
file_size recover=8
file_size replay=8
key=1, value=10
